# MoMA Online Archive Scrape

![img](img/banner.png)

This project uses `urllib` and `BeautifulSoup` libraries to scrape data from the Museum of Modern Art's [online collection](https://www.moma.org/collection/works?&with_images=1&on_view=1).

## Structure of Response Data

The relevant HTML section we want to extract data has the following structure:

```
<li> - class "grid-item--work"
└── <a> tag - link to artwork page
    ├── <div> 
    │   └── <picture> tag
    │       └── <img> tag
    └── <div> with artwork data
        └── <h3>
            ├── <span> - artist name
            ├── <span> - artwork name
            └── <span> - artwork date
```

The goal is to pull the artwork data inside the `h3` tag.

In [156]:
# Each artist's archive has an associated ID.
# Create a dictionary that contains a mapping between the artist name and their ID
ARTISTS = {
    "le corbusier": {
        "name": "Le Corbusier",
        "id": 3426},
    "mies van der rohe": {
        "name": "Ludwig Mies van der Rohe",
        "id": 7166
    },
    "antoni gaudí": {
        "name": "Antoni Gaudí",
        "id": 2096
    },
}

In [157]:
import urllib
import re
from bs4 import BeautifulSoup
import numpy as np

class ExtraSoup(BeautifulSoup):
    """Child class of BeautifulSoup with additional methods"""
    
    """HTML Response Retrieval Helper Functions"""
    
    def get_artist_info(self, artist_info):
        """Initialize artist and artist_id attributes"""
        self.artist_name = artist_info['name']
        self.artist_id = artist_info['id']
    
    def build_url(self, query):
        """Build a url to query MoMA website"""
        url = (f'https://www.moma.org/artists/{query}?locale=en&page=1')
        return url

    def get_url_response(self, url):
        """Get HTML response from url"""
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            return response.read()

    def make_soup(self):
        """Create a BeautifulSoup class from HTML response"""
        self.html = BeautifulSoup(
            self.get_url_response(self.build_url(self.artist_id)),
            'html.parser'
        )
        
    """HTML Parsing Helper Functions"""
    
    def get_artwork_year(self, string):
        """Extra year string from HTML tag.
        Year data may come various forms like "1932-34", "1923-1924", "c.1934".
        Not a perfect solution, but in this case get the first instance of a valid
        year (ie. "1923-1924" would extract 1923)
        
        Reference: https://developers.google.com/edu/python/regular-expressions
        """
        result = re.search(r'\d{4}', string).group(0)
        return int(result)

    def validate_artist(self, metadata):
        """Validate whether artwork was made by artist we are querying for,
        since search results may return a variety of artists.
        """
        if metadata:
            if self.artist_name.lower() in metadata.lower():
                return True
        return False

    def validate_string(self, metadata):
        """Validate whether string exists and return a cleaned string"""
        # Some titles are wrapped in an <em> tag inside span
        if metadata.find("em"):
            return metadata.find("em").string
        elif metadata.string is not None:
            # Span tag contents are listed with lots of spaces and newlines
            return metadata.string.strip()
        return None

    def get_artwork_data(self):
        """Parse through HTML tags and extract artwork date"""
        artist_works = self.html.find_all('li', 'grid-item--work')
        self.artworks = []
        for i, artwork in enumerate(artist_works):
            artwork_dict = {}
            # get span tags that contain artwork data
            artwork_metadata = artwork.find_all("span")
            artist = self.validate_string(artwork_metadata[0])
            if self.validate_artist(artist):
                artwork_dict['index'] = i
                artwork_dict['artist'] = artist
                artwork_dict['title'] = self.validate_string(artwork_metadata[1])
                artwork_dict['year'] = self.get_artwork_year(
                    self.validate_string(artwork_metadata[2])
                )
            self.artworks.append(artwork_dict)
            
    def get_artwork_urls(self):
        """Extract URL of each artwork from the artist page"""
        artist_works = self.html.find_all('li', 'grid-item--work')
        self.artwork_urls = []
        for artwork in artist_works:
            url = artwork.find('a', 'grid-item__link', href=True)
            self.artwork_urls.append(url['href'])
    
    """Functions for analysis"""
    
    def get_artwork_year_median(self):
        """Get the median year of all the artworks"""
        years = []
        for work in self.artworks:
            years.append(work['year'])
        return int(np.median(years))
    
    def get_artwork_count(self):
        """Get a count of how many artworks were found in the online collection"""
        return len(self.artworks)
    
    """Todo: Add export functions (to JSON, CSV)"""

## Le Courbusier

<img src="img/cb_savoye.jpg" width="500"/>

In [158]:
le_corbusier = ExtraSoup()
le_corbusier.get_artist_info(ARTISTS['le corbusier'])
le_corbusier.make_soup()
le_corbusier.get_artwork_data()
le_corbusier.artworks

[{'index': 0,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret)',
  'title': 'Bol, Pipes, et Papiers Enroulés',
  'year': 1919},
 {'index': 1,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret)',
  'title': 'Still Life',
  'year': 1920},
 {'index': 2,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret)',
  'title': "L'Esprit Nouveau letterhead (Letter to Naum Gabo from Paul Dermée)",
  'year': 1920},
 {'index': 3,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret)',
  'title': 'Still Life with Objects',
  'year': 1923},
 {'index': 4,
  'artist': 'Alfred Roth, Le Corbusier (Charles-Édouard Jeanneret)',
  'title': 'Bed Frame',
  'year': 1927},
 {'index': 5,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret), Pierre Jeanneret',
  'title': 'Les Terrasses, Villa Stein-de-Monzie, Garches, France',
  'year': 1926},
 {'index': 6,
  'artist': 'Le Corbusier (Charles-Édouard Jeanneret), Pierre Jeanneret, Charlotte Perriand',
  'title': 'Chaise Longue (LC/4)',
  'year': 1928},
 {'in

## Ludwig Mies van der Rohe

<img src="img/mies_barcelona.jpg" width="500"/>

In [159]:
mies_van_der_rohe = ExtraSoup()
mies_van_der_rohe.get_artist_info(ARTISTS['mies van der rohe'])
mies_van_der_rohe.make_soup()
mies_van_der_rohe.get_artwork_data()
mies_van_der_rohe.artworks

[{'index': 0,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Bismarck Monument Project, Bingen, Germany, Elevation',
  'year': 1910},
 {'index': 1,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Bismarck Monument Project, Bingen, Germany (Perspective)',
  'year': 1910},
 {'index': 2,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Bismarck Monument, project, Bingen, Germany, Perspective view of courtyard',
  'year': 1910},
 {'index': 3,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Werner House, Berlin-Zehlendorf,  Germany, Four elevations',
  'year': 1913},
 {'index': 4,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Werner House, Berlin-Zehlendorf, Germany (Four elevations, section)',
  'year': 1913},
 {'index': 5,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Werner House, Berlin-Zehlendorf, Germany, Four plans',
  'year': 1913},
 {'index': 6,
  'artist': 'Ludwig Mies van der Rohe',
  'title': 'Werner House, Berlin-Zehlendorf, Germany, Four elevations,

## Antoni Gaudí 

<img src="img/gaudi_sagrada.jpg" width="500"/>

In [160]:
antoni_gaudi = ExtraSoup()
antoni_gaudi.get_artist_info(ARTISTS['antoni gaudí'])
antoni_gaudi.make_soup()
antoni_gaudi.get_artwork_data()
antoni_gaudi.artworks

[{'index': 0,
  'artist': 'Antoni Gaudí',
  'title': 'Floor Tiles from the Casa Mila, Barcelona',
  'year': 1905},
 {'index': 1,
  'artist': 'Antoni Gaudí',
  'title': 'Grille from the Casa Milá (La Pedrera), Barcelona, Spain',
  'year': 1906},
 {'index': 2, 'artist': 'Antoni Gaudí', 'title': 'Prayer Bench', 'year': 1898},
 {'index': 3,
  'artist': 'Antoni Gaudí',
  'title': 'Church of the Sagrada Familia, Barcelona, Spain (Model of a column)',
  'year': 1883},
 {'index': 4,
  'artist': 'Antoni Gaudí',
  'title': 'Church of the Sagrada Familia, Barcelona, Spain (Model of a nave window)',
  'year': 1883},
 {'index': 5, 'artist': 'Antoni Gaudí', 'title': 'Side Chair', 'year': 1905},
 {'index': 6,
  'artist': 'Aurèlia Muñoz, Antoni Gaudí',
  'title': 'Study of a catenary arch for the Gaudí crypt at Colonia Güell',
  'year': 1996}]

## Summary

In [161]:
def print_artwork_stats(artist):
    print(f'{artist.artist_name}: \n\
          1. Total artworks found in the collection: {artist.get_artwork_count()} \n\
          2. Median year of the artist\'s artworks in the collection: {artist.get_artwork_year_median()}')

In [162]:
artists = [le_corbusier, mies_van_der_rohe, antoni_gaudi]
for artist in artists:
    print_artwork_stats(artist)

Le Corbusier: 
          1. Total artworks found in the collection: 77 
          2. Median year of the artist's artworks in the collection: 1959
Ludwig Mies van der Rohe: 
          1. Total artworks found in the collection: 100 
          2. Median year of the artist's artworks in the collection: 1925
Antoni Gaudí: 
          1. Total artworks found in the collection: 7 
          2. Median year of the artist's artworks in the collection: 1905


## Week 10 - Additional Work: Scrape Individual Artwork Pages

The relevant HTML section we want to extract data has the following structure:

```
<section> - class "grid-item--work"
└── <dl> tag - class "work__caption"
    ├── <dt> - class "work__caption__term"
    │   └── <span> class "work__caption__term__text"
    ├── <dd> - class "work__caption__description"
    │   └── <span> - class "work__caption__description__text"
    │
    │ ... <dt><dd> pair continues for each description
    |   (e.g dimension, publisher, printer, edition, etc)
```

__The goal is to pull the artwork data inside the `dt` and `dd` tag as dictionary key/value pairs.__

In [231]:
class ArtworkSoup(BeautifulSoup):   
    def build_url(self, query):
        """Build a url to query MoMA website"""
        url = (f'https://www.moma.org/{query}')
        return url

    def get_url_response(self, url):
        """Get HTML response from url"""
        req = urllib.request.Request(url)
        with urllib.request.urlopen(req) as response:
            return response.read()
    
    def make_soup(self, queries):
        """Create a BeautifulSoup class from HTML response"""
        self.htmls = []
        for query in queries:
            self.htmls.append(
                BeautifulSoup(
                    self.get_url_response(self.build_url(query)),
                    'html.parser'
                )
            )
    
    def create_artwork_dict(self, index, terms, descriptions):
            artwork_dict = dict(zip(terms, descriptions))
            artwork_dict['index'] = index
            return artwork_dict
        
    def get_artwork_data(self):
        """Parse through HTML tagsk extract artwork data and create a list of dictionaries"""
        artwork_data = []
        for i, html in enumerate(self.htmls):
            terms = map(self.clean_html,
                        html.find_all('span',
                                      'work__caption__term__text'))
            descriptions = map(self.clean_html, 
                               html.find_all('span',
                                             'work__caption__description__text'))
            artwork_data.append(self.create_artwork_dict(i, terms, descriptions))
        self.artwork_data = artwork_data
        
    def clean_html(self, html):
        """Strip tag and newline characters"""
        # Some descriptions have <em> tags within the <span> tag.
        if html.find('em'):
            return html.find('em').string.strip() 
        return html.string.strip()

## Le Corbusier

In [234]:
# Get list of artwork page urls from artist page
le_corbusier.get_artwork_urls()

# Create BeautifulSoup child class and scrape data
c_artworks = ArtworkSoup()
c_artworks.make_soup(le_corbusier.artwork_urls)
c_artworks.get_artwork_data()

# Check a sample from the scraped data
c_artworks.artwork_data[0]

{'Medium': 'Pencil on paper',
 'Dimensions': '17 1/2 x 21 1/2" (44.5 x 54.6 cm)',
 'Credit': 'The Riklis Collection of McCrory Corporation',
 'Object number': '914.1983',
 'Copyright': '© 2020 Artists Rights Society (ARS), New York / ADAGP, Paris / FLC',
 'Department': 'Drawings and Prints',
 'index': 0}

## Mies van Der Rohe

In [168]:
# Get list of artwork page urls from artist page
mies_van_der_rohe.get_artwork_urls()

# Create BeautifulSoup child class and scrape data
m_artworks = ArtworkSoup()
m_artworks.make_soup(mies_van_der_rohe.artwork_urls)
m_artworks.get_artwork_data()

# Check a sample from the scraped data
m_artworks.artwork_data[0]

{'Medium': 'Pencil and colored pastel on tracing paper',
 'Dimensions': '39 3/4 x 7\' 1 1/2" (101 x 217.2 cm)',
 'Credit': 'Mies van der Rohe Archive, gift of the architect',
 'Object number': 'MR24.1',
 'Copyright': '© 2020 Artists Rights Society (ARS), New York / VG Bild-Kunst, Bonn',
 'Department': 'Architecture and Design',
 'index': 0}

## Gaudi

In [187]:
antoni_gaudi.artwork_urls

['/collection/works/2636?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/3844?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/2141?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/82402?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/82400?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/1888?artist_id=2096&locale=en&page=1&sov_referrer=artist',
 '/collection/works/288378?artist_id=2096&locale=en&page=1&sov_referrer=artist']

In [186]:
# Get list of artwork page urls from artist page
antoni_gaudi.get_artwork_urls()

# Create BeautifulSoup child class and scrape data
gaudi_artworks = ArtworkSoup()
gaudi_artworks.make_soup(antoni_gaudi.artwork_urls)
gaudi_artworks.get_artwork_data()

# Check a sample from the scraped data
gaudi_artworks.artwork_data[0]

{'Manufacturer': 'Escofet y Ca, Barcelona',
 'Medium': 'Ceramic',
 'Dimensions': 'Each: 9 7/8 x 7/8" (25.1 x 2.2 cm)',
 'Credit': 'Marshall Cogan Purchase Fund',
 'Object number': '203.1987.1-7',
 'Department': 'Architecture and Design',
 'index': 0}

### Merging two dictionaries

Now we have an `ExtraSoup()` object with the artist's artist name / artwork title / year, and `ArtworkSoup()` with extra artwork data.

We can merge them together with Pandas.

In [236]:
import pandas as pd

le_courbusier_df = pd.DataFrame(le_corbusier.artworks)
le_corbusier_artwork_df = pd.DataFrame(c_artworks.artwork_data)
le_corbusier_merged = le_courbusier_df.merge(le_corbusier_artwork_df, left_on='index', right_on='index')
le_corbusier_merged.drop('index', axis=1)

,artist,title,year,Medium,Dimensions,Credit,Object number,Copyright,Department,Manufacturer,Collaborating artist,Model maker,Printer,Engineer,Fabricator,Delineator,Publisher,Edition,Type,Portfolio
0,Le Corbusier (Charles-Édouard Jeanneret),"Bol, Pipes, et Papiers Enroulés",1919,Pencil on paper,"17 1/2 x 21 1/2"" (44.5 x 54.6 cm)",The Riklis Collection of McCrory Corporation,914.1983,"© 2020 Artists Rights Society (ARS), New York ...",Drawings and Prints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Le Corbusier (Charles-Édouard Jeanneret),Still Life,1920,Oil on canvas,"31 7/8 x 39 1/4"" (80.9 x 99.7 cm)",Van Gogh Purchase Fund,261.1937,"© 2020 Artists Rights Society (ARS), New York ...",Painting and Sculpture,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Le Corbusier (Charles-Édouard Jeanneret),L'Esprit Nouveau letterhead (Letter to Naum Ga...,1920,Letterpress,"11 7/8 × 8 3/8"" (30.2 × 21.3 cm)","Elaine Lustig Cohen Collection, gift of Lawren...",1230.2015,NaN,Architecture and Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Le Corbusier (Charles-Édouard Jeanneret),Still Life with Objects,1923,Conté crayon on paper,"26 1/2 x 37 5/8"" (66.5 x 95.5 cm)",Gift of Lily Auchincloss,303.1989,"© 2020 Artists Rights Society (ARS), New York ...",Drawings and Prints,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Alfred Roth, Le Corbusier (Charles-Édouard Jea...",Bed Frame,1927,Tubular nickel-plated brass and steel,"frame (.a): 6 x 35 5/8 x 73 3/4"" (15.2 x 90.5 ...",Gift of Curt Valentin,550.1953.a-c,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,"Thonet Frères, Paris, France",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,"Richard Pare, Le Corbusier (Charles-Édouard Je...","Unité d'Habitation, Marseille, France (View of...",2011,Digital color coupler laser prints,"36 1/2 x 72"" (92.7 x 182.9 cm)",Gift of Elise Jaffe + Jeffrey Brown,687.2013,NaN,Architecture and Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,"Richard Pare, Le Corbusier (Charles-Édouard Je...","Villa Savoye, Poissy-sur-Seine, France (Exteri...",2012,Digital color coupler laser prints,"32 3/8 x 73 1/4"" (82.2 x 186.1 cm)",Gift of Elise Jaffe + Jeffrey Brown,686.2013,NaN,Architecture and Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,"Richard Pare, Le Corbusier (Charles-Édouard Je...","Chapel de Notre-Dame-du-Haut, Ronchamp, France...",2012,Digital color coupler laser prints,"22 1/4 x 96"" (56.5 x 243.8 cm)",Gift of Elise Jaffe + Jeffrey Brown,688.2013,NaN,Architecture and Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,"Richard Pare, Le Corbusier (Charles-Édouard Je...","Assembly Hall, Chandigarh, India (Interior view)",2012,Digital color coupler laser prints,"45 x 72"" (114.3 x 182.9 cm)",Gift of Elise Jaffe + Jeffrey Brown,689.2013,NaN,Architecture and Design,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
mies_van_der_rohe_df = pd.DataFrame(mies_van_der_rohe.artworks)
mies_van_der_rohe_artwork_df = pd.DataFrame(m_artworks.artwork_data)
mies_merged = mies_van_der_rohe_df.merge(mies_van_der_rohe_artwork_df, left_on='index', right_on='index')
mies_merged.drop('index', axis=1)

,artist,title,year,Medium,Dimensions,Credit,Object number,Copyright,Department,Model maker
0,Ludwig Mies van der Rohe,"Bismarck Monument Project, Bingen, Germany, El...",1910,Pencil and colored pastel on tracing paper,"39 3/4 x 7' 1 1/2"" (101 x 217.2 cm)","Mies van der Rohe Archive, gift of the architect",MR24.1,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
1,Ludwig Mies van der Rohe,"Bismarck Monument Project, Bingen, Germany (Pe...",1910,Photo collage and ink on illustration board,"30 x 40"" (76.2 x 101.6 cm)","Mies van der Rohe Archive, gift of the architect",MR24.3,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
2,Ludwig Mies van der Rohe,"Bismarck Monument, project, Bingen, Germany, P...",1910,Gouache on linen,"55 1/2 x 94 1/2"" (141 x 240 cm)","Rob Beyer Purchase Fund, Edward Larrabee Barne...",192.1998,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
3,Ludwig Mies van der Rohe,"Werner House, Berlin-Zehlendorf, Germany, Fou...",1913,Print,"12 1/4 x 28 1/2"" (31.1 x 72.4 cm)","Mies van der Rohe Archive, gift of the architect",MR29.1,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
4,Ludwig Mies van der Rohe,"Werner House, Berlin-Zehlendorf, Germany (Four...",1913,Print,"12 1/4 x 29 1/4"" (31.1 x 74.3 cm)","Mies van der Rohe Archive, gift of the architect",MR29.2,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
...,...,...,...,...,...,...,...,...,...,...
95,Ludwig Mies van der Rohe,"Weissenhof Apartment House, ""The Dwelling"" Exh...",1926,Pencil on tracing paper,"38 x 33 3/4"" (96.5 x 85.7 cm)","Mies van der Rohe Archive, gift of the architect",MR4.145,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
96,Ludwig Mies van der Rohe,"Weissenhof Apartment House, ""The Dwelling"" Exh...",1926,Pencil on tracing paper,"41 x 20 1/2"" (104.1 x 52.1 cm)","Mies van der Rohe Archive, gift of the architect",MR4.149,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
97,Ludwig Mies van der Rohe,"Weissenhof Apartment House, ""The Dwelling"" Exh...",1926,Pencil and colored pencil on tracing paper,"39 × 45"" (99.1 × 114.3 cm)","Mies van der Rohe Archive, gift of the architect",MR4.167,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN
98,Ludwig Mies van der Rohe,"Weissenhof Housing Colony Master Plan, ""The Dw...",1926,Diazotype with colored pencil and charcoal,"17 1/4 x 21"" (43.8 x 53.4 cm)","Mies van der Rohe Archive, gift of the architect",MR4.195,"© 2020 Artists Rights Society (ARS), New York ...",Architecture and Design,NaN


In [188]:
gaudi_df = pd.DataFrame(antoni_gaudi.artworks)
gaudi_artwork_df = pd.DataFrame(gaudi_artworks.artwork_data)
gaudi_merged = gaudi_df.merge(gaudi_artwork_df, left_on='index', right_on='index')
gaudi_merged.drop('index', axis=1)

,artist,title,year,Manufacturer,Medium,Dimensions,Credit,Object number,Department
0,Antoni Gaudí,"Floor Tiles from the Casa Mila, Barcelona",1905,"Escofet y Ca, Barcelona",Ceramic,"Each: 9 7/8 x 7/8"" (25.1 x 2.2 cm)",Marshall Cogan Purchase Fund,203.1987.1-7,Architecture and Design
1,Antoni Gaudí,"Grille from the Casa Milá (La Pedrera), Barcel...",1906,NaN,Wrought iron,"65 3/4 x 73 3/4 x 19 5/8"" (167 x 187.3 x 49.8 cm)",Gift of H. H. Hecht in honor of George B. Hes...,456.1974,Architecture and Design
2,Antoni Gaudí,Prayer Bench,1898,Gaudí's Workshop,Spanish oak and wrought iron,"32 5/8 x 44 1/2 x 26"" (82.9 x 113 x 66 cm), se...",Estée and Joseph Lauder Design Fund,107.1976,Architecture and Design
3,Antoni Gaudí,"Church of the Sagrada Familia, Barcelona, Spai...",1883,NaN,Plaster,"55 3/4 x 6 1/4"" (141.6 x 15.9 cm)",Purchase,MC 60,Architecture and Design
4,Antoni Gaudí,"Church of the Sagrada Familia, Barcelona, Spai...",1883,NaN,Plaster,"10' 4"" x 29 1/2"" x 14 1/2"" (315 x 74.9 x 36.8 cm)",Purchase,MC 59.a-b,Architecture and Design
5,Antoni Gaudí,Side Chair,1905,"Casa Ribas Seva, Barcelona",Oak,"29 5/8 x 20 3/4 x 15 1/4"" (75.2 x 52.7 x 38.7 ...",Purchase,SC56.1977,Architecture and Design
6,"Aurèlia Muñoz, Antoni Gaudí",Study of a catenary arch for the Gaudí crypt a...,1996,NaN,String and metal,"20 7/8 × 14 3/16 × 14 3/16"" (53 × 36 × 36 cm)",Committee on Architecture and Design Funds,841.2018,Architecture and Design


## Export Dataframes

In [238]:
le_corbusier_merged.to_csv('le_corbusier.tsv', index=False, sep='\t')
mies_merged.to_csv('mies_van_der_rohe.tsv', index=False, sep='\t')
gaudi_merged.to_csv('antoni_gaudi.tsv', index=False, sep='\t')